In [1]:
import random

In [2]:
class Cell():
    '''Класс ячейки игрового поля.'''
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.cell_state = 'free'
        self.is_shoot_in = 'nay'
        
    @property
    def state(self):
        '''Возвращает статус, свободна ячейка или занята своим кораблем,
        скрытым кораблем противника (ил же является контуром корабля) 
        (free/ship/hidden).
        '''
        return self.cell_state
    
    @state.setter
    def change_state(self, new_state):
        '''Изменяет статус ячейки (free/ship/hidden).'''        
        self.cell_state = new_state
        
    @property
    def shoot_in(self):
        '''Возвращает значение, можно ли использовать ли ячейку для стрельбы 
        (nay/yep).
        '''        
        return self.is_shoot_in
    
    @shoot_in.setter
    def change_shoot_in(self, new_value):
        '''Изменяет значение ячейки для стрельбы (nay/yep).'''        
        self.is_shoot_in = new_value
        
    def __repr__(self):
        '''Возвращает название ячейки.'''
        return f'({self.x}, {self.y})'
    

In [3]:
class Board():
    '''Класс для создания и управления состоянием игровой доски.'''
    def __init__(self, player):
        '''Инициализирует новую доску (6x6) для игрока.'''
        self.player = player
        self.game_board = []
        for row in range(0, 6):
            new_row = []
            for col in range(0, 6):
                new_cell = Cell(row, col)
                new_row.append(new_cell)
            self.game_board.append(new_row)
    
    @property       
    def board(self):
        '''Выводит игровое поле с координатами и условными обозначениями.'''
        print(f'\n\tИгрок: {self.player}')
        print(f'\t  1 2 3 4 5 6', end='')
        for index, row in enumerate(self.game_board):
            print(f'\n\t{index + 1} ', end = '')
            for cell in row:
                if cell.state == 'free' and cell.shoot_in == 'nay':
                    print('.', end=' ')
                elif cell.state == 'free' and cell.shoot_in == 'yep':
                    print('*', end=' ')
                elif cell.state == 'ship' and cell.shoot_in == 'nay':
                    print('#', end=' ')
                elif cell.state == 'ship' and cell.shoot_in == 'yep':
                    print('X', end=' ')
                elif cell.state == 'hidden' and cell.shoot_in == 'nay':
                    print('.', end=' ')
                elif cell.state == 'hidden' and cell.shoot_in == 'yep':
                    print('X', end=' ')


In [4]:
class Ship():
    '''Класс для управления состоянием кораблей в игре.'''
    def __init__(self, player, level, name, cells_ship):
        self.player = player
        if self.player == 'human':
            self.board = human_board
        else:
            self.board = ai_board
        self.name = name
        self.ship_level = level
        self.ship_lives = level
        self.ship_status = 'alive'
        self.cells_ship = cells_ship
        if level > 1:
            for cell in cells_ship:
                x, y = cell
                if self.player == 'human':
                    self.board.game_board[x][y].change_state = 'ship'
                else:
                    self.board.game_board[x][y].change_state = 'hidden'
        else:
            x, y = cells_ship
            if self.player == 'human':
                self.board.game_board[x][y].change_state = 'ship'
            else:
                self.board.game_board[x][y].change_state = 'hidden'
         
    @property
    def lives(self):
        '''Возвращает уровень живучести корабля.'''
        return self.ship_lives
        
    def check_ship(self, cell):
        '''Изменяет уровень живучести корабля при попадании в его ячейку.'''
        if cell in self.cells_ship:
            self.ship_lives -= 1
            return True
        else:
            return False
  
    @property
    def status(self):
        '''Изменяет статус корабля, утратившего свою живучесть.'''
        if self.lives == 0:
            self.ship_status = 'dead'        
        return self.ship_status
    
    def __repr__(self):
        '''Возвращает название корабля'''
        return self.name


In [5]:
class User():
    '''Класс для определения игрока и управления стрельбой.'''
    def __init__(self, type):
        '''Инициализирует игрока, устанавливает его тип (human/ai)'''
        self.type = type
        
    @property
    def shoot(self):
        '''Устанавливает кординаты обстреливаемой ячейки'''
        pass
        
    def __repr__(self):
        '''Возвращает имя игрока'''
        return self.name
    
class Human(User):
    '''Класс для определения игрока-человека и управления стрельбой.'''
    def __init__(self, type):
        '''Инициализирует игрока, устанавливает его имя и тип (human)'''
        super().__init__(type)
        
    @property
    def shoot(self):
        '''Устанавливает координаты обстреливаемой ячейки.'''
        print('Введите цель для выстрела - координаты формата xy,')
        print('где x - номер строки, y - номер столбца (например, 01 или 33)')
        choice = input('Ваш выбор: ')
        if len(choice) < 2 or len(choice) > 2:
            print('Координаты должны быть в формате xy, где x и y - целые числа')
            return False
        try:    
            x = int(choice[0])
            y = int(choice[1])
            if x > 0 and x < 7:
                if y > 0 and y < 7:
                    x -= 1
                    y -= 1
                    if ai_board.game_board[x][y].shoot_in != 'yep':
                        ai_board.game_board[x][y].is_shoot_in = 'yep'
                        self.shot = (x, y)
                        return True
                    else:
                        print('Сюда уже стреляли!')
                        return False
                else:
                    print('Координаты должны быть целыми числами в диапазоне 1-6')
                    return False
            else:
                print('Координаты должны быть целыми числами в диапазоне 1-6')
                return False
        except ValueError:
            print('Это не координаты!')
            return False
            
class AI(User):
    '''Класс для определения AI-игрока-человека и управления стрельбой.'''
    def __init__(self, type):
        '''Инициализирует игрока, устанавливает его имя и тип (human)'''
        super().__init__(type)
        
    @property
    def shoot(self):
        '''Устанавливает координаты обстреливаемой ячейки.'''
        x = random.randint(0, 5)
        y = random.randint(0, 5)
        if human_board.game_board[x][y].shoot_in != 'yep':
            human_board.game_board[x][y].is_shoot_in = 'yep'
            self.shot = (x, y)
            return True
        else:
            return False 


In [6]:
human_board = Board('human')

In [7]:
ai_board = Board('ai')

In [8]:
human = Human('human')

In [9]:
ai = AI('ai')

In [10]:
human.shoot

Введите цель для выстрела - координаты формата xy,
где x - номер строки, y - номер столбца (например, 01 или 33)


Ваш выбор:  11


True

In [11]:
while human.shoot is not True:
    human.shoot

Введите цель для выстрела - координаты формата xy,
где x - номер строки, y - номер столбца (например, 01 или 33)


Ваш выбор:  


Координаты должны быть в формате xy, где x и y - целые числа
Введите цель для выстрела - координаты формата xy,
где x - номер строки, y - номер столбца (например, 01 или 33)


Ваш выбор:  ab


Это не координаты!
Введите цель для выстрела - координаты формата xy,
где x - номер строки, y - номер столбца (например, 01 или 33)


Ваш выбор:  111


Координаты должны быть в формате xy, где x и y - целые числа
Введите цель для выстрела - координаты формата xy,
где x - номер строки, y - номер столбца (например, 01 или 33)


Ваш выбор:  11


Сюда уже стреляли!
Введите цель для выстрела - координаты формата xy,
где x - номер строки, y - номер столбца (например, 01 или 33)


Ваш выбор:  22


In [14]:
ai.shoot

True

In [15]:
ai.shot

(3, 1)

In [25]:
human_board.game_board[3][1].shoot_in

'yep'

In [24]:
ai_board.game_board[1][1].shoot_in

'yep'

In [17]:
class Squadron():
    '''Класс устанавливает игровое поле и управляет состоянием всех кораблей игрока.'''
    def __init__(self, player):
        '''Инициализирует расстановку кораблей на доске.'''
        self.player = player
        self.ships = []
        self.player_scores = 7
            
    def place_ships(self):
        '''Расставляет корабли из паттернов.'''
        pattern_1 = [((3, 0), (4, 0), (5, 0)),
                     ((0, 1), (1, 1)), ((1, 3), (1, 4)),
                     (3, 2), (3, 4), (5, 3), (5, 5)]
        pattern_2 = [((2, 2), (2, 3), (2, 4)),
                     ((0, 4), (0, 5)), ((5, 0), (5, 1)),
                     (0, 2), (1, 0), (4, 3), (4, 5)]
        choice = random.randint(1, 2)
        if choice == 1:
            user_pattern = pattern_1
        else:
            user_pattern = pattern_2
        ship_3 = Ship(self.player, 3, 'ship_3', user_pattern[0])
        self.ships.append(ship_3)
        ship_2_1 = Ship(self.player, 2, 'ship_2_1', user_pattern[1])
        self.ships.append(ship_2_1)
        ship_2_2 = Ship(self.player, 2, 'ship_2_2', user_pattern[2])
        self.ships.append(ship_2_2)
        ship_1_1 = Ship(self.player, 1, 'ship_1_1', user_pattern[3])
        self.ships.append(ship_1_1)
        ship_1_2 = Ship(self.player, 1, 'ship_1_2', user_pattern[4])
        self.ships.append(ship_1_2)
        ship_1_3 = Ship(self.player, 1, 'ship_1_3', user_pattern[5])
        self.ships.append(ship_1_3)
        ship_1_4 = Ship(self.player, 1, 'ship_1_4', user_pattern[6])
        self.ships.append(ship_1_4)
        
    @property
    def scores(self):
        '''Возвращает счет оставшихся кораблей игрока.'''
        return self.player_scores
    
    @scores.setter
    def check_scores(self):
        '''Уменьшает счет всех кораблей эскадры при гибели корабля.'''
        for ship in self.ships:
            if ship.status == 'dead':
                self.player_scores += 1
        

In [18]:
human_squadron = Squadron('human')

In [19]:
human_squadron.place_ships()

In [20]:
human_squadron.ships

[ship_3, ship_2_1, ship_2_2, ship_1_1, ship_1_2, ship_1_3, ship_1_4]

In [21]:
human_board.game_board[3][0].state

'free'

In [22]:
human_board.board


	Игрок: human
	  1 2 3 4 5 6
	1 . . # . # # 
	2 # . . . . . 
	3 . . # # # . 
	4 . * . . . . 
	5 . . . # . # 
	6 # # . . . . 

In [26]:
ai_board.board


	Игрок: ai
	  1 2 3 4 5 6
	1 * . . . . . 
	2 . * . . . . 
	3 . . . . . . 
	4 . . . . . . 
	5 . . . . . . 
	6 . . . . . . 